In [1]:
import pandas as pd
import requests

In [2]:
def get_reviews(appid, params):
        url_start = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url_start+appid, params=params)
        return response.json() # return data extracted from the json response

In [4]:
appid = '1091500' # Cyberpunk 2077, good mix of positive and negative, just for samples
cursor = '*'
params = { # https://partner.steamgames.com/doc/store/getreviews
        'json' : 1,
        'filter' : 'all', # sort by: recent, updated, all (helpfullness)
        'language' : 'english', # https://partner.steamgames.com/doc/store/localization
        'day_range' : 9223372036854775807, # shows reveiws from all time
        'cursor' : cursor.encode(), # for pagination
        'review_type' : 'all', # all, positive, negative
        'purchase_type' : 'all', # all, non_steam_purchase, steam
        'num_per_page' : 100 # max amount per request
    }

In [19]:
printProgress=True
n=2000

In [20]:
results = []
for i in range(n//100):
    result = get_reviews(appid, params)
    results += result['reviews']
    params['cursor'] = result['cursor']

In [24]:
reviews_df = pd.DataFrame(results)[['review', 'voted_up']]
reviews_df

,review,voted_up
0,While it does feel like they needed a bit more...,True
1,Game is asbolutely good. The Night City is som...,True
2,This game has a JoJo reference.,True
3,"Cheers everyone, after 8 years we finally made...",True
4,made my penis to perfection in a call with fri...,True
...,...,...
1995,The game doesn't bring anything new to the tab...,False
1996,pp go smol ( ͡° ͜ʖ ͡°)\n\npp go big (˵ ͡☉ ͜ʖ ͡...,True
1997,"Great characters, nice city, thrilling storyli...",True
1998,So here is my review after all of this time.\n...,True


In [25]:
reviews_df['voted_up'].value_counts()

True     1207
False     793
Name: voted_up, dtype: int64

In [28]:
reviews_df.to_csv('data/test_reviews.csv', index=False)

In [9]:
get_all_reviews(appid, params, printProgress=True)

NameError: name '_makeRequest' is not defined

In [8]:
def get_all_reviews(appid, params, printProgress=False, n=0):
    results = pd.DataFrame()

    data = _makeRequest(appid, params)
    done = False

    while not done:
        if 'success' in data and data['success'] == 1: # if the query was successful
            if 'reviews' in data and len(data['reviews']) > 0: # if we received reviews
                results += data['reviews'] # add the reviews in this query to our results
                params['start_offset'] += data['query_summary']['num_reviews'] # increase the start offset by the number of reviews received in this response
                if printProgress:
                    print('{amount} reviews found...'.format(amount=params['start_offset']))
                data = _makeRequest(appid, params) # get the next page of reviews
            else: # there are no more reviews
                done = True
        elif data is None: # Steam Web API returns null if rate limit is reached.
            done = True
            raise ConnectionRefusedError('Steam Web API returned null. Rate limit may be exceeded.')
        else: # unsuccessful API call raises an error
            done = True
            raise ConnectionError('Steam Web API appreviews request was unsuccessful.')
    
    if printProgress:
        print("Found all reviews.")
    return results